In [1]:
import numpy as np
import pandas as pd
from scipy.stats import spearmanr


from xgboost import XGBRegressor
from sklearn.linear_model import LassoCV
from sklearn.model_selection import TimeSeriesSplit


import warnings
warnings.filterwarnings("ignore")

In [2]:
sent = pd.read_csv("sentiment.csv", index_col=0, parse_dates=True)
rec  = pd.read_csv("recommendation.csv", index_col=0, parse_dates=True)
price = pd.read_csv("price.csv", index_col=0, parse_dates=True)

sent, rec = sent.align(rec, join="inner", axis=0)
sent, price = sent.align(price, join="inner", axis=0)
rec = rec.loc[sent.index]

In [3]:
# Sentiment features - multiple windows and transformations
f_sent1 = sent.add_suffix("_s1")
f_sent3 = sent.rolling(3).mean().add_suffix("_s3")
f_sent7 = sent.rolling(7).mean().add_suffix("_s7")
f_sent14 = sent.rolling(14).mean().add_suffix("_s14")
f_sent_std = sent.rolling(7).std().add_suffix("_s_std")  # Sentiment volatility
f_sent_change = sent.diff().add_suffix("_s_change")  # Change in sentiment

# Recommendation features
f_rec1 = rec.add_suffix("_r1")
f_rec3 = rec.rolling(3).mean().add_suffix("_r3")
f_rec7 = rec.rolling(7).mean().add_suffix("_r7")
f_rec_change = rec.diff().add_suffix("_r_change")  # Change in recommendations

# Price-based momentum features
returns = price.pct_change(fill_method=None)
mom1 = returns.add_suffix("_mom1")
mom5 = price.pct_change(5, fill_method=None).add_suffix("_mom5")
mom10 = price.pct_change(10, fill_method=None).add_suffix("_mom10")
mom21 = price.pct_change(21, fill_method=None).add_suffix("_mom21")
mom63 = price.pct_change(63, fill_method=None).add_suffix("_mom63")  # 3-month momentum

# Volatility features
vol5 = returns.rolling(5).std().add_suffix("_vol5")
vol10 = returns.rolling(10).std().add_suffix("_vol10")
vol21 = returns.rolling(21).std().add_suffix("_vol21")

# RSI-like features (relative strength)
def rsi_like(series, window=14):
    """RSI-like indicator: (avg gain - avg loss) / (avg gain + avg loss)"""
    delta = series.pct_change(fill_method=None)
    gain = delta.where(delta > 0, 0).rolling(window).mean()
    loss = -delta.where(delta < 0, 0).rolling(window).mean()
    rs = gain / (loss + 1e-10)
    return (rs - 1) / (rs + 1)

rsi14 = price.apply(lambda x: rsi_like(x, 14)).add_suffix("_rsi14")
rsi7 = price.apply(lambda x: rsi_like(x, 7)).add_suffix("_rsi7")

# Mean reversion features
price_ma5 = price.rolling(5).mean()
price_ma21 = price.rolling(21).mean()
price_ma63 = price.rolling(63).mean()
zscore5 = ((price - price_ma5) / (price.rolling(5).std() + 1e-10)).add_suffix("_zscore5")
zscore21 = ((price - price_ma21) / (price.rolling(21).std() + 1e-10)).add_suffix("_zscore21")

# Relative features (stock vs market)
market_sent = sent.mean(axis=1)
market_rec = rec.mean(axis=1)
market_ret = returns.mean(axis=1)

sent_rel = sent.sub(market_sent, axis=0).add_suffix("_sent_rel")
rec_rel = rec.sub(market_rec, axis=0).add_suffix("_rec_rel")
ret_rel = returns.sub(market_ret, axis=0).add_suffix("_ret_rel")

In [4]:
# Combine all features
features = pd.concat([
    f_sent1, f_sent3, f_sent7, f_sent14, f_sent_std, f_sent_change,
    f_rec1, f_rec3, f_rec7, f_rec_change,
    mom1, mom5, mom10, mom21, mom63,
    vol5, vol10, vol21,
    rsi7, rsi14,
    zscore5, zscore21,
    sent_rel, rec_rel, ret_rel
], axis=1)

# Lag all features by 1 day to avoid look-ahead bias
# This ensures we use yesterday's features to predict tomorrow's return
features_lagged = features.shift(1)

# Target: next day's return
returns = price.pct_change(fill_method=None).shift(-1)

# Debug: Check feature statistics
print(f"Features shape: {features_lagged.shape}")
print(f"Features with NaN: {features_lagged.isna().sum().sum()}")
print(f"Features non-NaN count: {features_lagged.notna().sum().sum()}")
print(f"Sample feature columns: {list(features_lagged.columns[:5])}")


Features shape: (270, 1268)
Features with NaN: 55471
Features non-NaN count: 286889
Sample feature columns: ['1COVG.DE_s1', 'AALB.AS_s1', 'ABB.ST_s1', 'ABI.BR_s1', 'ABNd.AS_s1']


In [5]:
# Reshape features for modeling
X = features_lagged.stack().reset_index()
X.columns = ["date", "col", "value"]

# Extract RIC and feature name from column names
# Handle multi-part feature names (e.g., "AAPL_sent_rel")
X["ric"] = X["col"].str.split("_").str[0]
# Join remaining parts as feature name
X["feature_name"] = X["col"].str.split("_", n=1).str[1]

X = X.drop(columns=["col"])

X = X.pivot_table(index=["date","ric"], 
                  columns="feature_name", 
                  values="value")

y = returns.stack().reset_index()
y.columns = ["date", "ric", "target"]
y = y.set_index(["date","ric"])

# Align X and y on index before joining
df = X.join(y, how="inner").dropna()

print(f"\nAfter joining and dropping NaN:")
print(f"DataFrame shape: {df.shape}")
print(f"Target stats: mean={df['target'].mean():.6f}, std={df['target'].std():.6f}")
print(f"Target NaN count: {df['target'].isna().sum()}")
print(f"Feature columns: {len([c for c in df.columns if c != 'target'])}")

# Remove extreme outliers (beyond 3 standard deviations) - more efficient approach
feature_cols = [col for col in df.columns if col != "target"]
mask = pd.Series(True, index=df.index)

for col in feature_cols:
    mean = df[col].mean()
    std = df[col].std()
    if std > 0:  # Avoid division by zero
        mask = mask & (df[col] >= mean - 3*std) & (df[col] <= mean + 3*std)

df = df[mask]

print(f"\nAfter outlier removal:")
print(f"DataFrame shape: {df.shape}")
print(f"Remaining samples: {len(df)}")

X_mat = df.drop(columns=["target"])
y_vec = df["target"]

# Final check
print(f"\nFinal X_mat shape: {X_mat.shape}")
print(f"X_mat NaN count: {X_mat.isna().sum().sum()}")
print(f"X_mat constant columns: {(X_mat.nunique() == 1).sum()}")
print(f"y_vec NaN count: {y_vec.isna().sum()}")
print(f"y_vec stats: mean={y_vec.mean():.6f}, std={y_vec.std():.6f}")


After joining and dropping NaN:
DataFrame shape: (6491, 26)
Target stats: mean=0.000304, std=0.028934
Target NaN count: 0
Feature columns: 25

After outlier removal:
DataFrame shape: (5705, 26)
Remaining samples: 5705

Final X_mat shape: (5705, 25)
X_mat NaN count: 0
X_mat constant columns: 1
y_vec NaN count: 0
y_vec stats: mean=0.000361, std=0.022659


## LASSO REGRESSION

In [6]:
model = LassoCV(cv=5, n_alphas=100)
model.fit(X_mat, y_vec)

pred = model.predict(X_mat)
df["predicted_return"] = pred

market_ret = returns.mean(axis=1)
df["market"] = df.index.get_level_values("date").map(market_ret)
df["alpha"] = df["predicted_return"] - df["market"]

In [7]:
ic = np.corrcoef(df["predicted_return"], df["target"])[0,1]
print("IC:", ic)

IC: 0.01508442677701171


In [8]:
accuracy = ((df["predicted_return"] > 0) 
            == (df["target"] > 0)).mean()

print("Directional accuracy:", accuracy)


Directional accuracy: 0.5123575810692375


## XGBOOST

In [9]:
tscv = TimeSeriesSplit(n_splits=5)

preds = []
actuals = []
fold_ics = []

print("Training XGBOOST with improved features and hyperparameters...")
print("-" * 60)

for fold, (train_idx, test_idx) in enumerate(tscv.split(X_mat), 1):
    X_train, X_test = X_mat.iloc[train_idx], X_mat.iloc[test_idx]
    y_train, y_test = y_vec.iloc[train_idx], y_vec.iloc[test_idx]
    
    # Check for issues
    if len(X_train) == 0 or len(X_test) == 0:
        print(f"Fold {fold}: Empty train or test set!")
        continue
    
    # Remove constant features
    constant_cols = X_train.columns[X_train.nunique() == 1]
    if len(constant_cols) > 0:
        X_train = X_train.drop(columns=constant_cols)
        X_test = X_test.drop(columns=constant_cols)
    
    if X_train.shape[1] == 0:
        print(f"Fold {fold}: No valid features after removing constants!")
        continue
    
    # Further split training data for validation
    split_point = int(len(X_train) * 0.9)
    if split_point == 0:
        split_point = len(X_train)
    X_train_fit, X_val = X_train.iloc[:split_point], X_train.iloc[split_point:]
    y_train_fit, y_val = y_train.iloc[:split_point], y_train.iloc[split_point:]

    # Use a reasonable fixed number of estimators with strong regularization
    # This avoids early stopping compatibility issues across XGBoost versions
    xgb = XGBRegressor(
        n_estimators=500,  # Fixed number - regularization prevents overfitting
        max_depth=5,  # Slightly deeper for more complex patterns
        learning_rate=0.01,  # Lower learning rate for better generalization
        subsample=0.85,  # Slightly higher subsample
        colsample_bytree=0.85,  # Feature subsampling
        colsample_bylevel=0.85,  # Additional regularization
        min_child_weight=3,  # Regularization
        gamma=0.1,  # Minimum loss reduction for splits
        reg_alpha=0.1,  # L1 regularization
        reg_lambda=1.0,  # L2 regularization
        objective="reg:squarederror",
        random_state=42,
        n_jobs=-1
    )
    
    # Fit model - use validation set for monitoring but don't rely on early stopping
    try:
        if len(X_val) > 0:
            xgb.fit(
                X_train_fit, y_train_fit,
                eval_set=[(X_val, y_val)],
                verbose=False
            )
        else:
            xgb.fit(X_train_fit, y_train_fit, verbose=False)
    except Exception as e:
        print(f"Fold {fold}: Error fitting model: {e}")
        continue
    
    best_iter = 500  # Fixed number of estimators used
    
    fold_preds = xgb.predict(X_test)
    preds.extend(fold_preds)
    actuals.extend(y_test)
    
    # Calculate IC with robust NaN handling
    fold_preds_array = np.array(fold_preds)
    y_test_array = np.array(y_test)
    
    # Remove any NaN values
    valid_mask = ~(np.isnan(fold_preds_array) | np.isnan(y_test_array))
    pred_std = 0.0
    target_std = 0.0
    
    if valid_mask.sum() < 2:
        fold_ic = np.nan
        fold_ic_spearman = np.nan
    else:
        fold_preds_clean = fold_preds_array[valid_mask]
        y_test_clean = y_test_array[valid_mask]
        
        # Check for sufficient variance
        pred_std = np.std(fold_preds_clean)
        target_std = np.std(y_test_clean)
        
        if pred_std > 1e-10 and target_std > 1e-10 and len(fold_preds_clean) > 1:
            try:
                # Try Pearson correlation first
                fold_ic = np.corrcoef(fold_preds_clean, y_test_clean)[0, 1]
                if np.isnan(fold_ic) or np.isinf(fold_ic):
                    # Fall back to Spearman
                    fold_ic = spearmanr(fold_preds_clean, y_test_clean)[0]
                    fold_ic_spearman = fold_ic
                else:
                    # Also calculate Spearman for comparison
                    fold_ic_spearman = spearmanr(fold_preds_clean, y_test_clean)[0]
            except Exception as e:
                fold_ic = np.nan
                fold_ic_spearman = np.nan
        else:
            fold_ic = np.nan
            fold_ic_spearman = np.nan
    
    fold_ics.append(fold_ic)
    
    # Print detailed fold information
    if not np.isnan(fold_ic):
        print(f"Fold {fold} IC: {fold_ic:.6f} (Spearman: {fold_ic_spearman:.6f}, test_size: {len(X_test)}, valid: {valid_mask.sum()})")
    else:
        print(f"Fold {fold} IC: NaN (test_size: {len(X_test)}, pred_std: {pred_std:.6f}, target_std: {target_std:.6f}, valid: {valid_mask.sum()})")

print("-" * 60)
if len(preds) > 1 and np.std(preds) > 0 and np.std(actuals) > 0:
    ic_oos = np.corrcoef(preds, actuals)[0, 1]
    if np.isnan(ic_oos):
        ic_oos = spearmanr(preds, actuals)[0]
else:
    ic_oos = np.nan

print(f"Overall IC out-of-sample: {ic_oos:.6f}")
valid_fold_ics = [ic for ic in fold_ics if not np.isnan(ic)]
if len(valid_fold_ics) > 0:
    print(f"Mean fold IC: {np.mean(valid_fold_ics):.6f}")
    print(f"Std fold IC: {np.std(valid_fold_ics):.6f}")
else:
    print("Mean fold IC: NaN (all folds had NaN IC)")

# Calculate additional metrics
preds_array = np.array(preds)
actuals_array = np.array(actuals)
if len(preds_array) > 0:
    directional_accuracy = ((preds_array > 0) == (actuals_array > 0)).mean()
    print(f"Directional accuracy: {directional_accuracy:.4f}")
    
    # Rank IC (correlation of ranks)
    if len(preds_array) > 1:
        rank_ic = spearmanr(preds_array, actuals_array)[0]
        print(f"Rank IC (Spearman): {rank_ic:.6f}")
    
    # Debug info
    print(f"\nPredictions stats: mean={np.mean(preds_array):.6f}, std={np.std(preds_array):.6f}")
    print(f"Actuals stats: mean={np.mean(actuals_array):.6f}, std={np.std(actuals_array):.6f}")
    print(f"Unique predictions: {len(np.unique(preds_array))}")

Training XGBOOST with improved features and hyperparameters...
------------------------------------------------------------
Fold 1 IC: NaN (test_size: 950, pred_std: 0.000000, target_std: 0.028672, valid: 950)
Fold 2 IC: NaN (test_size: 950, pred_std: 0.000000, target_std: 0.020184, valid: 950)
Fold 3 IC: NaN (test_size: 950, pred_std: 0.000000, target_std: 0.023320, valid: 950)
Fold 4 IC: NaN (test_size: 950, pred_std: 0.000000, target_std: 0.019372, valid: 950)
Fold 5 IC: NaN (test_size: 950, pred_std: 0.000000, target_std: 0.022443, valid: 950)
------------------------------------------------------------
Overall IC out-of-sample: -0.035068
Mean fold IC: NaN (all folds had NaN IC)
Directional accuracy: 0.4920
Rank IC (Spearman): -0.042846

Predictions stats: mean=0.000418, std=0.000480
Actuals stats: mean=0.000057, std=0.023148
Unique predictions: 5


In [10]:
# Feature importance analysis
# Train a final model on all data (except last fold) to get feature importance
final_train_idx = list(tscv.split(X_mat))[-1][0]  # Use all but last fold for training
X_train_final = X_mat.iloc[final_train_idx].copy()
y_train_final = y_vec.iloc[final_train_idx].copy()

# Remove constant features
constant_cols = X_train_final.columns[X_train_final.nunique() == 1]
if len(constant_cols) > 0:
    print(f"Removing {len(constant_cols)} constant features for feature importance analysis")
    X_train_final = X_train_final.drop(columns=constant_cols)

if X_train_final.shape[1] == 0:
    print("No valid features for feature importance analysis!")
else:
    split_point = int(len(X_train_final) * 0.9)
    if split_point == 0:
        split_point = len(X_train_final)
    X_train_fit_final, X_val_final = X_train_final.iloc[:split_point], X_train_final.iloc[split_point:]
    y_train_fit_final, y_val_final = y_train_final.iloc[:split_point], y_train_final.iloc[split_point:]

    xgb_final = XGBRegressor(
        n_estimators=500,
        max_depth=5,
        learning_rate=0.01,
        subsample=0.85,
        colsample_bytree=0.85,
        colsample_bylevel=0.85,
        min_child_weight=3,
        gamma=0.1,
        reg_alpha=0.1,
        reg_lambda=1.0,
        objective="reg:squarederror",
        random_state=42,
        n_jobs=-1
    )

    # Fit model
    try:
        if len(X_val_final) > 0:
            xgb_final.fit(
                X_train_fit_final, y_train_fit_final,
                eval_set=[(X_val_final, y_val_final)],
                verbose=False
            )
        else:
            xgb_final.fit(X_train_fit_final, y_train_fit_final, verbose=False)
        
        # Get feature importance
        feature_importance = pd.DataFrame({
            'feature': X_train_final.columns,
            'importance': xgb_final.feature_importances_
        }).sort_values('importance', ascending=False)

        print("\nTop 20 Most Important Features:")
        print("=" * 60)
        print(feature_importance.head(20).to_string(index=False))
        
        # Show features with zero importance
        zero_importance = feature_importance[feature_importance['importance'] == 0]
        if len(zero_importance) > 0:
            print(f"\nFeatures with zero importance: {len(zero_importance)}")
    except Exception as e:
        print(f"Error in feature importance analysis: {e}")


Removing 1 constant features for feature importance analysis

Top 20 Most Important Features:
 feature  importance
    mom1         0.0
   mom10         0.0
zscore21         0.0
    vol5         0.0
   vol21         0.0
   vol10         0.0
sent_rel         0.0
   s_std         0.0
s_change         0.0
      s7         0.0
      s3         0.0
     s14         0.0
      s1         0.0
    rsi7         0.0
   rsi14         0.0
 ret_rel         0.0
 rec_rel         0.0
      r7         0.0
      r3         0.0
      r1         0.0

Features with zero importance: 24
